In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# Quick Start

In [ ]:
# ---- Construct S.A.M. with default settings ----
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
# ---- Choose frequencies, and calculate GWB ----
fobs = utils.nyquist_freqs(15.0*YR, 0.1*YR)
print(utils.stats(1/(fobs*YR)), "[yr]")

In [ ]:
gwb = sam.gwb(fobs, realize=10)    # calculate 10 different realizations

In [ ]:
# ---- Plot GWB ----

fig, ax = plt.subplots(figsize=[8, 4])
ax.set(xscale='log', xlabel='Frequency [1/yr]', yscale='log', ylabel='Characteristic Strain')
ax.grid(alpha=0.2)

xx = fobs * YR
xx = kale.utils.midpoints(xx) 
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)
ax.plot(xx, np.median(gwb, axis=-1), 'k-')
for pp in [50, 98]:
    percs = pp / 2
    percs = [50 - percs, 50 + percs]
    ax.fill_between(xx, *np.percentile(gwb, percs, axis=-1), alpha=0.25, color='b')

plt.show()

# build SAM component-by-component

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_MM13() # M-MBulge Relation            (MMB)

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)

In [ ]:
# ---- Calculate GWB Amplitude distribution at 1/yr ----
fobs = 1.0/YR
fobs = [0.9*fobs, 1.1*fobs]
ayr = sam.gwb(fobs, realize=100)
print(ayr.shape, utils.stats(ayr))

In [ ]:
fig, ax = plt.subplots(figsize=[8, 4])
ax.set(xlabel=r'$\log_{10}(A_\mathrm{yr})$', ylabel='Probability Density')
ax.grid(alpha=0.2)
kale.dist1d(ayr, density=True, confidence=True)

plt.show()

## Plot GWB Amplitude Distribution vs. M-MBulge parameters

In [ ]:
alpha_list = [0.85, 1.0, 1.15]
norm_list = np.logspace(7, 9, 5)
NREALS = 100
# FREQ = 1.0 / YR   # [1/s]

dist_mmb = np.zeros((len(alpha_list), norm_list.size, NREALS))

for aa, alpha in enumerate(tqdm.tqdm(alpha_list)):
    for nn, norm in enumerate(tqdm.tqdm(norm_list, leave=False)):
        mmbulge = holo.relations.MMBulge_Standard(mamp=norm*MSOL, mplaw=alpha)
        sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=(21, 20, 19))
        # dist_mmb[aa, nn, :] = sam.gwb(FREQ, realize=NREALS)
        dist_mmb[aa, nn, :] = sam.gwb(fobs, realize=NREALS)

In [ ]:
fig, ax = plt.subplots(figsize=[6, 4])
ax.set(xscale='log', xlabel='M-MBulge Mass', yscale='log', ylabel=r'GWB $A_\mathrm{yr}$')
ax.grid(alpha=0.2)

dist = dist_mmb

for aa, dd in zip(alpha_list, dist):
    med = np.median(dd, axis=-1)
    cc, = ax.plot(norm_list, med, label=aa)
    cc = cc.get_color()
    ax.fill_between(norm_list, *np.percentile(dd, [25, 75], axis=-1), color=cc, alpha=0.25)
    
plt.legend(title='slope')
plt.show()